In [2]:
import numpy as np
import pandas as pd

In [3]:
def getNAs(df):

    colNames = []
    percentNA = []
    for i in df.columns:
        colNames.append(i)
        numNA = df[i].isna().sum()
        percent = (numNA/len(df))*100
        percentNA.append(percent)
        
    colNames = pd.DataFrame(colNames)
    colNames = colNames.rename(columns={0: "label"})
    percentNA = pd.DataFrame(percentNA)
    percentNA = percentNA.rename(columns={0: "numNA"})
    d = pd.concat([colNames,percentNA], axis = 1).sort_values(by=['numNA'], ascending = False)

        
    return d

In [4]:
def remove_columns(df, threshold_percent):
    '''drop columns by a threshold (percentage of na)'''
    data = getNAs(df)
    column_names = data[data.numNA >= threshold_percent].label
    clean_data = df.drop(column_names, axis = 1)
    
    return clean_data





In [5]:
#convert all categories to lowercase, there is an issue with factor levels of y, n, N, Y for yes/no 
def entry_to_lowercase(df):
    for i in df.columns:
        if (df[i].dtype == "O"):
            df[i] = semi_clean_crop[i].str.lower()
    return df

In [6]:
### identify which are categorical and which are numeric by looking at the factor levels
## create dataframe with variable name, levels, number of categories and data types

def id_data_types(df):
    names = pd.DataFrame()
    category = pd.DataFrame()
    data_type = pd.DataFrame()
    numUni = pd.DataFrame()

    for i in df.columns:
        
        names = names.append({"variable": i},ignore_index = True)
        category = category.append({"category": list(df[i].unique())}, ignore_index = True)
        numUni = numUni.append({"numUnique": len(df[i].unique())}, ignore_index = True)
        data_type = data_type.append({"data_type": df[i].dtype}, ignore_index = True)
    
    view_data = pd.concat([names, category, numUni,data_type],axis =1)
    
    return (view_data)

In [7]:
def replace_na_with_NaN(df):
    for i in df.columns:
        df[i] = df[i].replace('na', np.NaN)
    return df



In [8]:
rhomis_data = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/RHoMIS_Full_Data.csv", engine = "python")
rh_indic = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/RHoMIS_Indicators.csv", engine='python')

In [9]:
## final dataset, keep and drop these columns
rhomis_data = rhomis_data[["crop_count", "crop_name_1", "crop_harvest_1", "crop_intercrop_1"]]
rh_indic = rh_indic.drop(['ITERATION','GPS_LAT', 'GPS_LON', 'GPS_ALT', 'Altitude','WorstFoodSecMonth',
       'BestFoodSecMonth','FIES_Score',  'currency_conversion_factor', 'GHGEmissions' ],axis = 1)

In [10]:
bigData  = pd.concat([rhomis_data,rh_indic ],axis = 1)
bigData = remove_columns(bigData, 50)
bigData.head(3)


bigData = replace_na_with_NaN(bigData)
bigData.dtypes


crop_count                                       object
crop_name_1                                      object
crop_harvest_1                                   object
crop_intercrop_1                                 object
ID_PROJ                                          object
ID_COUNTRY                                       object
YEAR                                              int64
SURVEY_ID                                        object
ID_HH                                            object
Country                                          object
HHsizemembers                                     int64
HHsizeMAE                                       float64
HouseholdType                                    object
Head_EducationLevel                              object
LandOwned                                       float64
LandCultivated                                  float64
LivestockHoldings                               float64
NrofMonthsFoodInsecure                          

In [11]:
a = bigData.select_dtypes('object')
id_data_types(a.select_dtypes('object'))

bigData.crop_count = bigData.crop_count.fillna(-500)
bigData.crop_count = bigData.crop_count.astype(int)
bigData.crop_count = bigData.crop_count.replace(-500, np.nan)

In [12]:
bigData["continent"] = 0
bigData['Country'].unique()

array(['Tanzania', 'Guatemala', 'Honduras', 'El_Salvador', 'Mali',
       'Burkina_Faso', 'Malawi', 'Kenya', 'India', 'Cambodia', 'Vietnam',
       'LaoPDR', 'Ethiopia', 'DRC', 'Zambia', 'Ghana', 'Uganda',
       'Costa_Rica', 'Burundi', 'Peru', 'Nicaragua'], dtype=object)

In [14]:
bigData.continent[bigData.Country == 'Tanzania']

0        africa
1        africa
2        africa
3        africa
4        africa
          ...  
10831    africa
10832    africa
10833    africa
10834    africa
10835    africa
Name: continent, Length: 2508, dtype: object

In [13]:
bigData.continent[bigData.Country == 'Tanzania'] = "africa"


bigData.continent[bigData.Country == "Guatemala"] = "central_america"
bigData.continent[bigData.Country == "Honduras"] = "central_america"
bigData.continent[bigData.Country == "El_Salvador"] = "central_america"
bigData.continent[bigData.Country == "Mali"] = "africa"
bigData.continent[bigData.Country == "Burkina_Faso"] = "africa"
bigData.continent[bigData.Country == "Malawi"] = "africa"
bigData.continent[bigData.Country == "Kenya"] = "africa"
bigData.continent[bigData.Country == "India"] = "asia"
bigData.continent[bigData.Country == "Cambodia"] = "asia"
bigData.continent[bigData.Country == "Vietnam"] = "asia"
bigData.continent[bigData.Country == "LaoPDR"] = "asia"
bigData.continent[bigData.Country == "Ethiopia"] = "africa"
bigData.continent[bigData.Country == "DRC"] = "africa"
bigData.continent[bigData.Country == "Zambia"] = "africa"
bigData.continent[bigData.Country == "Ghana"] = "africa"
bigData.continent[bigData.Country == "Uganda"] = "africa"
bigData.continent[bigData.Country == "Costa_Rica"] = "central_america"
bigData.continent[bigData.Country == "Burundi"] = "africa"
bigData.continent[bigData.Country == "Peru"] = "south_america"
bigData.continent[bigData.Country == "Nicaragua"] = "central_america"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [16]:
bigData.crop_intercrop_1.unique() #category 

array([nan, 'intercrop', 'monoculture'], dtype=object)

In [18]:
bigData.crop_harvest_1.unique() #category

array([nan, 'bad_harvest', 'good_harvest', 'normal_harvest'], dtype=object)

In [22]:
bigData.YEAR.unique() # category 

array([2015, 2016, 2017, 2018])

In [26]:
# convert these to categories
bigData.crop_intercrop_1 = bigData.crop_intercrop_1.astype('category')

bigData.crop_harvest_1 = bigData.crop_harvest_1.astype('category')

bigData.YEAR = bigData.YEAR.astype('category')

In [27]:
bigData.dtypes

crop_count                                       float64
crop_name_1                                       object
crop_harvest_1                                  category
crop_intercrop_1                                category
ID_PROJ                                           object
ID_COUNTRY                                        object
YEAR                                            category
SURVEY_ID                                         object
ID_HH                                             object
Country                                           object
HHsizemembers                                      int64
HHsizeMAE                                        float64
HouseholdType                                     object
Head_EducationLevel                               object
LandOwned                                        float64
LandCultivated                                   float64
LivestockHoldings                                float64
NrofMonthsFoodInsecure         

In [28]:
# Write to CSV: only needs to be run once.
bigData.to_csv("data/clean.csv", encoding='utf-8', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/clean.csv'